<a href="https://colab.research.google.com/github/wenshanliu-slu0/hds5210-2023/blob/main/week11/week11_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 11 Assignment


Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `https://hds5210-data.s3.amazonaws.com/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [24]:
import pandas as pd
# This is just to show you the name to use for the variable you need to create for this step to pass.
all_hospitals = pd.read_csv('https://hds5210-data.s3.amazonaws.com/complications_all.csv')


In [25]:
# Do you work here and in as many cells as you need to create a variable called `mo_summary` that matches the requirements

#Q0
# Filter data for hospitals in MO
mo_hospitals = all_hospitals[all_hospitals['State'] == 'MO']

#print(mo_hospitals)

#Q1
# Convert 'Start Date' and 'End Date' to datetime fields
mo_hospitals['Start Date'] = pd.to_datetime(mo_hospitals['Start Date'])
mo_hospitals['End Date'] = pd.to_datetime(mo_hospitals['End Date'])

#Q2
# clean up and convert the Denominator field to just be numeric - the rule that you should use it to simply remove any records where the Denominator is 'Not Available'
mo_hospitals = mo_hospitals[mo_hospitals['Denominator'] != 'Not Available']
mo_hospitals['Denominator'] = pd.to_numeric(mo_hospitals['Denominator'])

print(mo_hospitals)


      Facility ID                       Facility Name  \
45534      260001               MERCY HOSPITAL JOPLIN   
45535      260001               MERCY HOSPITAL JOPLIN   
45536      260001               MERCY HOSPITAL JOPLIN   
45537      260001               MERCY HOSPITAL JOPLIN   
45538      260001               MERCY HOSPITAL JOPLIN   
...           ...                                 ...   
47557      261337  MISSOURI BAPTIST SULLIVAN HOSPITAL   
47558      261337  MISSOURI BAPTIST SULLIVAN HOSPITAL   
47575      261338             MERCY HOSPITAL CARTHAGE   
47576      261338             MERCY HOSPITAL CARTHAGE   
47577      261338             MERCY HOSPITAL CARTHAGE   

                         Address      City State  ZIP Code County Name  \
45534              100 MERCY WAY    JOPLIN    MO     64804      JASPER   
45535              100 MERCY WAY    JOPLIN    MO     64804      JASPER   
45536              100 MERCY WAY    JOPLIN    MO     64804      JASPER   
45537              

<ipython-input-25-a646065f45d6>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['Start Date'] = pd.to_datetime(mo_hospitals['Start Date'])
<ipython-input-25-a646065f45d6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mo_hospitals['End Date'] = pd.to_datetime(mo_hospitals['End Date'])


In [26]:
#final step
#how to using NameAgg() https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.NamedAgg.html


# Aggregate data
mo_summary = mo_hospitals.groupby('Facility Name').agg(
    start_date=pd.NamedAgg(column='Start Date', aggfunc='min'),
    end_date=pd.NamedAgg(column='End Date', aggfunc='max'),
    number=pd.NamedAgg(column='Denominator', aggfunc='sum')
).reset_index()


mo_summary.set_index('Facility Name', inplace=True)

# Display the resulting DataFrame
print(mo_summary)



                                    start_date   end_date  number
Facility Name                                                    
BARNES JEWISH HOSPITAL              2015-04-01 2018-06-30  131313
BARNES-JEWISH ST PETERS HOSPITAL    2015-04-01 2018-06-30   15668
BARNES-JEWISH WEST COUNTY HOSPITAL  2015-04-01 2018-06-30    9622
BATES COUNTY MEMORIAL HOSPITAL      2015-07-01 2018-06-30    3117
BELTON REGIONAL MEDICAL CENTER      2015-04-01 2018-06-30    9270
...                                        ...        ...     ...
TRUMAN MEDICAL CENTER LAKEWOOD      2015-04-01 2018-06-30    4297
UNIVERSITY OF MISSOURI HEALTH CARE  2015-04-01 2018-06-30   56493
WASHINGTON COUNTY MEMORIAL HOSPITAL 2015-07-01 2018-06-30     220
WESTERN MISSOURI MEDICAL CENTER     2015-04-01 2018-06-30    7254
WRIGHT MEMORIAL HOSPITAL            2015-07-01 2018-06-30     198

[108 rows x 3 columns]


In [27]:
assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

**Double-click to enter your answer**

- Relational Database
- Web Services
- local files
- Internet


#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

**Double-click to enter your answer**

- CSV
- JSON
- Excel
- HTML


#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

**Double-click to enter your answer**

Long-term care analysis is critical for a number of reasons, the most important of which are the aging population and the possible financial load associated with longer healthcare demands. Therefore, I want to do a visual analysis of long-term care facilities and nursing homes to gain a comprehensive understanding of the quality of care provided in our society.

In this report, Maryland will be used as an example. Firstly, data will be extracted using the Python package from https://www.indexmundi.com/ to obtain elderly population statistics for each county in Maryland. In the second part, data on Assisted Living will be extracted from data.gov using an API-based approach. In the final step of the data set retrieval process, information on Maryland Long Term Care Assisted Living, Acute General, and Special Hospitals will be collected. This data will be imported into a Python data frame using the local files approach, either through CSV or Excel files.

 Subsequently, these datasets will be joined if in the specific condition, which is the county column. Therefore, the final report would use those data to make a heatmap to show the higher value of the elderly population and a map to show the locations of assisted living places and acute and special hospitals.




---



## Submit your work via GitHub as normal
